In [ ]:
import numpy as np
import pandas as pd
import keras
import keras.backend as K
from keras.optimizers import Adam
from keras.models import Sequential
from keras.utils import Sequence
from keras.layers import *
import matplotlib.pyplot as plt
from google.colab import files
import io 
from google.colab import drive

In [ ]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/Projects 2021/Sudoku ML Solver/sudoku.csv"
df = pd.read_csv(path)

In [ ]:
df_sudoku = pd.DataFrame({"quizzes":df["puzzle"],
                          "solutions":df["solution"]})
df_sudoku.head(5)

,quizzes,solutions
0,0700000430400096108006349000940520003584600200...,6795182435437296188216349577943521863584617292...
1,3010865040465210705000000014008000020803479000...,3719865248465213795924738614638197522853479167...
2,0483015603600080909106700030200009355090102006...,7483915623652487919126754834217869355894132766...
3,0083170000042051090000400703271609049014500000...,2983176457642851391539462783271689549814537266...
4,0408906300001368208007405190004670524500207002...,1428956379751368248367425193984671524513287962...


In [ ]:
print("Quiz:\n",np.array(list(map(int,list(df_sudoku['quizzes'][0])))).reshape(9,9))
print("Solution:\n",np.array(list(map(int,list(df_sudoku['solutions'][0])))).reshape(9,9))

Quiz:
 [[0 7 0 0 0 0 0 4 3]
 [0 4 0 0 0 9 6 1 0]
 [8 0 0 6 3 4 9 0 0]
 [0 9 4 0 5 2 0 0 0]
 [3 5 8 4 6 0 0 2 0]
 [0 0 0 8 0 0 5 3 0]
 [0 8 0 0 7 0 0 9 1]
 [9 0 2 1 0 0 0 0 5]
 [0 0 7 0 4 0 8 0 2]]
Solution:
 [[6 7 9 5 1 8 2 4 3]
 [5 4 3 7 2 9 6 1 8]
 [8 2 1 6 3 4 9 5 7]
 [7 9 4 3 5 2 1 8 6]
 [3 5 8 4 6 1 7 2 9]
 [2 1 6 8 9 7 5 3 4]
 [4 8 5 2 7 6 3 9 1]
 [9 6 2 1 8 3 4 7 5]
 [1 3 7 9 4 5 8 6 2]]


In [ ]:
class DataGenerator(Sequence):
    def __init__(self, df,batch_size = 16,subset = "train",shuffle = False, info={}):
        super().__init__()
        self.df = df
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.subset = subset
        self.info = info
        
        self.data_path = path
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.floor(len(self.df)/self.batch_size))
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df))
        if self.shuffle==True:
            np.random.shuffle(self.indexes)
            
    def __getitem__(self,index):
        X = np.empty((self.batch_size, 9,9,1))
        y = np.empty((self.batch_size,81,1))
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        for i,f in enumerate(self.df['quizzes'].iloc[indexes]):
            self.info[index*self.batch_size+i]=f
            X[i,] = (np.array(list(map(int,list(f)))).reshape((9,9,1))/9)-0.5
        if self.subset == 'train': 
            for i,f in enumerate(self.df['solutions'].iloc[indexes]):
                self.info[index*self.batch_size+i]=f
                y[i,] = np.array(list(map(int,list(f)))).reshape((81,1)) - 1
        if self.subset == 'train': return X, y
        else: return X

In [ ]:
model = Sequential()

model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same', input_shape=(9,9,1)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(1,1), activation='relu', padding='same'))

model.add(Flatten())
model.add(Dense(81*9))
model.add(Reshape((-1, 9)))
model.add(Activation('softmax'))

adam = keras.optimizers.Adam(lr=.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 9, 9, 64)          640       
_________________________________________________________________
batch_normalization (BatchNo (None, 9, 9, 64)          256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 64)          36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 9, 64)          256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 128)         8320      
_________________________________________________________________
flatten (Flatten)            (None, 10368)             0         
_________________________________________________________________
dense (Dense)                (None, 729)               7

In [ ]:
train_idx = int(len(df_sudoku)*0.95)
df_sudoku = df_sudoku.sample(frac=1).reset_index(drop=True)
training_generator = DataGenerator(df_sudoku.iloc[:train_idx], subset = "train", batch_size=640)
validation_generator = DataGenerator(df_sudoku.iloc[train_idx:], subset = "train",  batch_size=640)

In [ ]:
training_generator.__getitem__(4)[0].shape, validation_generator.__getitem__(4)[0].shape

((640, 9, 9, 1), (640, 9, 9, 1))

In [ ]:
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau
filepath1="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
filepath2 = "best_weights.hdf5"
checkpoint1 = ModelCheckpoint(filepath1, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
checkpoint2 = ModelCheckpoint(filepath2, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    patience=3,
    verbose=1,
    min_lr=1e-6
)
callbacks_list = [checkpoint1,checkpoint2,reduce_lr]

In [ ]:
history = model.fit_generator(training_generator, 
                              validation_data = validation_generator, 
                              epochs = 1, 
                              verbose=1, 
                              callbacks=callbacks_list)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


13359/13359 [==============================] - 13489s 1s/step - loss: 0.5718 - accuracy: 0.1129 - val_loss: 0.3785 - val_accuracy: 0.1106

Epoch 00001: val_accuracy improved from -inf to 0.11057, saving model to weights-improvement-01-0.11.hdf5

Epoch 00001: val_accuracy improved from -inf to 0.11057, saving model to best_weights.hdf5


In [ ]:
model.load_weights('best_weights.hdf5')

Now, we will fill one number at a time, which means that, we will pass the sudoku through the neural network one-time, check if the number fits, and then fill it in. Again, we will pass the sudoku for another number, check and then fill.

This way, the neural network will get the help to gain context about the next vaccant number, based on previoulsy passed numbers.

In [ ]:
def norm(a):
    return (a/9)-.5

def denorm(a):
    return (a+.5)*9

In [ ]:
def inference_sudoku(sample):
    feat = sample
    
    while(1):
    
        out = model.predict(feat.reshape((1,9,9,1)))  
        out = out.squeeze()

        pred = np.argmax(out, axis=1).reshape((9,9))+1 
        prob = np.around(np.max(out, axis=1).reshape((9,9)), 2) 
        
        feat = denorm(feat).reshape((9,9))
        mask = (feat==0)
     
        if(mask.sum()==0):
            break
            
        prob_new = prob*mask
    
        ind = np.argmax(prob_new)
        x, y = (ind//9), (ind%9)

        val = pred[x][y]
        feat[x][y] = val
        feat = norm(feat)
    
    return pred

def test_accuracy(feats, labels):
    
    correct = 0
    
    for i,feat in enumerate(feats):
        
        pred = inference_sudoku(feat)
        
        true = labels[i].reshape((9,9))+1
        
        if(abs(true - pred).sum()==0):
            correct += 1
        
    print(correct/feats.shape[0])

In [ ]:
def solve_sudoku(game):
    
    game = game.replace('\n', '')
    game = game.replace(' ', '')
    game = np.array([int(j) for j in game]).reshape((9,9,1))
    game = norm(game)
    game = inference_sudoku(game)
    return game

In [ ]:
new_game = '''
          0 0 0 0 0 0 0 0 0
          0 0 0 0 0 0 0 0 0
          0 0 0 0 0 0 0 0 0
          0 0 0 0 0 0 0 0 0
          0 0 0 0 0 0 0 0 0
          0 0 0 0 0 0 0 0 0
          0 0 0 0 0 0 0 0 0
          0 0 0 0 0 0 0 0 0
          0 0 0 0 0 0 0 0 0
      '''

game = '''
          0 0 0 7 0 0 0 9 6
          0 0 3 0 6 9 1 7 8
          0 0 7 2 0 0 5 0 0
          0 7 5 0 0 0 0 0 0
          9 0 1 0 0 0 3 0 0
          0 0 0 0 0 0 0 0 0
          0 0 9 0 0 0 0 0 1
          3 1 8 0 2 0 4 0 7
          2 4 0 0 0 5 0 0 0
      '''

game = solve_sudoku(game)

print('solved puzzle:\n')
print(game)

solved puzzle:

[[1 8 4 7 5 3 2 9 6]
 [5 2 3 4 6 9 1 7 8]
 [6 9 7 2 1 8 5 4 3]
 [8 7 5 6 3 2 9 1 4]
 [9 6 1 5 4 7 3 8 2]
 [4 3 2 8 9 1 7 6 5]
 [7 5 9 3 8 4 6 2 1]
 [3 1 8 9 2 6 4 5 7]
 [2 4 6 1 7 5 8 3 9]]


In [ ]:
np.sum(game, axis=1)

array([45, 45, 45, 45, 45, 45, 45, 45, 45])

The Naked Twin approach is the faster way to solve Sudoku.

I refered the following article: Solving Sudoku using a naked twin strategy- [Visit Here!](https://medium.com/@anandpathak69/solving-sudoku-using-naked-twin-strategies-f7ed23ea867f)

In [ ]:
import collections

rows = 'ABCDEFGHI'
cols = '123456789'

def cross(A, B):
    "Cross product of elements in A and elements in B."
    return [s + t for s in A for t in B]


boxes = cross(rows, cols)
row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC', 'DEF', 'GHI') for cs in ('123', '456', '789')]
unitlist = row_units + column_units + square_units 
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s], [])) - set([s])) for s in boxes)


In [ ]:
def assign_value(values, box, value):
    values[box] = value
    return values


def naked_twins(values):
    
    for unit in unitlist:
    
        unit_values_counter = collections.Counter([values[box] for box in unit])
        for twins, count in unit_values_counter.items():
    
            if 1 < count == len(twins):
                for box in unit:
                    if values[box] != twins and set(values[box]).intersection(set(twins)):
                        for digit in twins:
                            values = assign_value(values, box, values[box].replace(digit, ''))
    return values


def grid_values(grid):

    chars = []
    digits = '123456789'
    for c in grid:
        if c in digits:
            chars.append(c)
        if c == '0':
            chars.append(digits)
    assert len(chars) == 81
    return dict(zip(boxes, chars))


def display(values):
    width = 1 + max(len(values[s]) for s in boxes)
    line = '+'.join(['-' * (width * 3)] * 3)
    for r in rows:
        print(''.join(values[r + c].center(width) + ('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    print

In [ ]:
def eliminate(values):
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    for box in solved_values:
        digit = values[box]
        for peer in peers[box]:
            values[peer] = values[peer].replace(digit, '')
    return values


def only_choice(values):
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                values[dplaces[0]] = digit
    return values


def reduce_puzzle(values):
    stalled = False
    while not stalled:
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])
        values = eliminate(values)
        values = only_choice(values)
        values = naked_twins(values)
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        stalled = solved_values_before == solved_values_after
        if len([box for box in values.keys() if len(values[box]) == 0]):
            #display(values)
            return False
    return values


def search(values):
    values = reduce_puzzle(values)
    if values is False:
        return False
    if all(len(values[s]) == 1 for s in boxes):
        return values
    
    min_possibility_box = min([box for box in boxes if len(values[box]) > 1])
    
    for digit in values[min_possibility_box]:
        new_sudoku = values.copy()
        new_sudoku[min_possibility_box] = digit
        attempt = search(new_sudoku)
        if attempt:
            return attempt

def solve2(grid):

    values = grid_values(grid)
    values = search(values)
    return values

In [ ]:
val_set = df_sudoku.iloc[:1000]


from tqdm import tqdm
quiz_list = list(val_set['quizzes'])
sol_list = list(val_set['solutions'])
val_quiz = []
val_sol = []
for i,j in tqdm(zip(quiz_list,sol_list)):
    q = np.array(list(map(int,list(i)))).reshape(9,9)
    s = np.array(list(map(int,list(j)))).reshape(9,9)
    val_quiz.append(q)
    val_sol.append(s)

1000it [00:00, 20979.49it/s]


In [ ]:
count = 0
for row in tqdm(df_sudoku.head(1000).iterrows()):
    if (solve2(row[1]["quizzes"]) == grid_values(row[1]["solutions"])):
        count+=1
        
print("Solved: {}/1,000 \nAccuracy: {}%.\n".format(count,(count/1000.0)*100))

1000it [00:03, 304.45it/s]

Solved: 1000/1,000 
Accuracy: 100.0%.



Conclusion:

* While we know Neural network perform better in generic task, they are not very accurate in solving the Sudoku.
* The best approach according to this experiment is Naked Twins approach. 
* There are more versions of this approach out there, which are even faster than the Naked Twins. They are called Naked Triplets and even Naked Quadruples.

Thank you!